In [12]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression

In [23]:
class MyTreeReg:
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20, bins=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0  # Добавляем переменную для хранения кол-ва листьев
        self.tree = None  # Здесь будем хранить дерево, которое создадим
        self.bins = bins  # Количество бинов для гистограмм
        self.fi = {}  # Важность фичей
        
    def __str__(self):
        params = vars(self) # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyTreeReg class: {params_str}"
    
    def mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def get_best_split(self, X, y):
        best_gain = -np.inf  # Изначально прирост установлен как отрицательная бесконечность
        best_split = {'col_name': None, 'split_value': None, 'gain': None}  # Инициализируем структуру для хранения лучшего сплита

        current_mse = self.mse(y)  # Текущий MSE для всего набора данных

        # Проходим по всем колонкам
        for col in X.columns:
            unique_values = np.sort(X[col].unique())  # Получаем уникальные отсортированные значения в колонке
            
             # Если количество уникальных значений меньше или равно bins, используем их
            if self.bins is None or len(unique_values) <= self.bins:
                split_candidates = (unique_values[:-1] + unique_values[1:]) / 2
            else:
                # Если больше, строим гистограмму
                _, bin_edges = np.histogram(X[col], bins=self.bins)
                split_candidates = (bin_edges[:-1] + bin_edges[1:]) / 2

            # Проходим по всем уникальным значениям (кроме последнего), чтобы выбрать разделитель
            for split_value in split_candidates:
                # Разделяем данные на две части
                left_mask = X[col] <= split_value
                right_mask = X[col] > split_value

                # Проверяем, что обе части содержат достаточно данных (учитываем min_samples_split)
                if left_mask.sum() < self.min_samples_split or right_mask.sum() < self.min_samples_split:
                    continue

                # Вычисляем MSE для каждой части
                left_mse = self.mse(y[left_mask])
                right_mse = self.mse(y[right_mask])

                # Вычисляем взвешенное MSE после разделения
                weighted_mse = (left_mask.sum() * left_mse + right_mask.sum() * right_mse) / len(y)

                # Вычисляем прирост (разницу) MSE
                gain = current_mse - weighted_mse

                # Если прирост больше текущего лучшего, обновляем лучший сплит
                if gain > best_gain:
                    best_gain = gain
                    best_split['col_name'] = col
                    best_split['split_value'] = split_value
                    best_split['gain'] = gain

        return best_split
    
    def fit(self, X, y):
        self.leafs_cnt = 0  # Сбрасываем счетчик листьев перед началом обучения
        self.fi = {col: 0 for col in X.columns}  # Инициализируем важность фичей нулями
        self.tree = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):
        n_samples, n_features = X.shape

        # Останавливаем рост дерева при достижении условий
        if depth >= self.max_depth or n_samples < self.min_samples_split or self.leafs_cnt >= self.max_leafs:
            self.leafs_cnt += 1
            return np.mean(y)  # Возвращаем среднее значение как результат для листа

        # Ищем лучший сплит
        best_split = self.get_best_split(X, y)

        if best_split['gain'] is None or best_split['gain'] == 0:
            self.leafs_cnt += 1
            return np.mean(y)
        
        # Обновляем важность фичи
        self.fi[best_split['col_name']] += best_split['gain']

        # Разделяем данные по лучшему сплиту
        left_mask = X[best_split['col_name']] <= best_split['split_value']
        right_mask = X[best_split['col_name']] > best_split['split_value']

        # Рекурсивно строим поддеревья
        left_tree = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        right_tree = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        # Возвращаем узел дерева
        return {
            'split_feature': best_split['col_name'],
            'split_value': best_split['split_value'],
            'left': left_tree,
            'right': right_tree
        }
        
    def feature_importances(self):
        return self.fi

    def print_tree(self, tree=None, depth=0):
        if tree is None:
            tree = self.tree

        # Если это лист, выводим его значение
        if not isinstance(tree, dict):
            print(f"{' ' * depth * 4}Leaf: {tree:.2f}")
            return

        # Иначе выводим сплит узел и рекурсивно отображаем поддеревья
        print(f"{' ' * depth * 4}Node: [Feature: {tree['split_feature']}, Split Value: {tree['split_value']:.2f}]")
        self.print_tree(tree['left'], depth + 1)
        self.print_tree(tree['right'], depth + 1)
        
    def predict_row(self, tree, row):
        # Рекурсивная функция для предсказания значения по одному ряду
        # Если это лист, возвращаем значение листа
        if not isinstance(tree, dict):
            return tree
        
        # Идем по дереву в зависимости от значения признака
        if row[tree['split_feature']] <= tree['split_value']:
            return self.predict_row(tree['left'], row)
        else:
            return self.predict_row(tree['right'], row)

    def predict(self, X):
        # Метод для предсказания на основе обученного дерева
        return [self.predict_row(self.tree, row) for _, row in X.iterrows()]

In [24]:
# Тестирование важности фичей
X1, y1 = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X1 = pd.DataFrame(X1, columns=[f'feature_{i}' for i in range(X1.shape[1])])
y1 = pd.Series(y1)

X2, y2 = make_regression(n_samples=150, n_features=3, noise=0.5, random_state=24)
X2 = pd.DataFrame(X2, columns=[f'feature_{i}' for i in range(X2.shape[1])])
y2 = pd.Series(y2)

tree = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=10, bins=5)

tree.fit(X1, y1)
feature_importances_1 = tree.feature_importances()
print(f"Важность фичей для первого набора данных: {feature_importances_1}\n")

tree.fit(X2, y2)
feature_importances_2 = tree.feature_importances()
print(f"Важность фичей для второго набора данных: {feature_importances_2}\n")

Важность фичей для первого набора данных: {'feature_0': 7201.791960173448, 'feature_1': 5307.428716540061}

Важность фичей для второго набора данных: {'feature_0': 3048.962741935009, 'feature_1': 10359.538573743625, 'feature_2': 7317.395889800144}



In [22]:
# Тестирование гистограмм
X1, y1 = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X1 = pd.DataFrame(X1, columns=[f'feature_{i}' for i in range(X1.shape[1])])
y1 = pd.Series(y1)

X2, y2 = make_regression(n_samples=150, n_features=3, noise=0.5, random_state=24)
X2 = pd.DataFrame(X2, columns=[f'feature_{i}' for i in range(X2.shape[1])])
y2 = pd.Series(y2)

tree = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=10, bins=5)

tree.fit(X1, y1)
predictions_1 = tree.predict(X1)
sum_predictions_1 = sum(predictions_1)
print(f"Сумма предсказаний для первого набора данных: {sum_predictions_1}\n")

tree.fit(X2, y2)
predictions_2 = tree.predict(X2)
sum_predictions_2 = sum(predictions_2)
print(f"Сумма предсказаний для второго набора данных: {sum_predictions_2}\n")

Сумма предсказаний для первого набора данных: -761.4847620370264

Сумма предсказаний для второго набора данных: -867.3195994201674



In [19]:
# Тестирование предсказания
X1, y1 = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X1 = pd.DataFrame(X1, columns=[f'feature_{i}' for i in range(X1.shape[1])])
y1 = pd.Series(y1)

X2, y2 = make_regression(n_samples=150, n_features=3, noise=0.5, random_state=24)
X2 = pd.DataFrame(X2, columns=[f'feature_{i}' for i in range(X2.shape[1])])
y2 = pd.Series(y2)

# Создаем экземпляр класса MyTreeReg
tree = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=10)


tree.fit(X1, y1)
predictions_1 = tree.predict(X1)
sum_predictions_1 = sum(predictions_1)
print(f"Сумма предсказаний для первого набора данных: {sum_predictions_1}\n")


tree.fit(X2, y2)
predictions_2 = tree.predict(X2)
sum_predictions_2 = sum(predictions_2)
print(f"Сумма предсказаний для второго набора данных: {sum_predictions_2}\n")

Сумма предсказаний для первого набора данных: -761.4847620370272

Сумма предсказаний для второго набора данных: -867.3195994201674



In [16]:
X1, y1 = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X1 = pd.DataFrame(X1, columns=[f'feature_{i}' for i in range(X1.shape[1])])
y1 = pd.Series(y1)

X2, y2 = make_regression(n_samples=120, n_features=5, noise=0.5, random_state=24)
X2 = pd.DataFrame(X2, columns=[f'feature_{i}' for i in range(X2.shape[1])])
y2 = pd.Series(y2)

tree = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=10)

# Обучение на первом наборе данных
tree.fit(X1, y1)
print("Dataset 1:")
print(f"Количество листьев для первого набора данных: {tree.leafs_cnt}\n")

# Обучение на втором наборе данных
tree.fit(X2, y2)
print("Dataset 2:")
print(f"Количество листьев для второго набора данных: {tree.leafs_cnt}")

Dataset 1:
Количество листьев для первого набора данных: 13

Dataset 2:
Количество листьев для второго набора данных: 12


In [10]:
# Тестирование лучшего сплита
X1, y1 = make_regression(n_samples=100, n_features=2, noise=0.1, random_state=42)
X2, y2 = make_regression(n_samples=150, n_features=3, noise=10, random_state=42)

# Преобразуем данные в pandas DataFrame и Series
X1 = pd.DataFrame(X1, columns=[f'feature_{i}' for i in range(X1.shape[1])])
y1 = pd.Series(y1)
X2 = pd.DataFrame(X2, columns=[f'feature_{i}' for i in range(X2.shape[1])])
y2 = pd.Series(y2)

# Создаем экземпляр класса MyTreeReg
tree = MyTreeReg(max_depth=5, min_samples_split=2, max_leafs=20)

# Датасет 1
best_split_1 = tree.get_best_split(X1, y1)
print(f"Best split: Feature = {best_split_1['col_name']}, Split Value = {best_split_1['split_value']}, Gain = {best_split_1['gain']}\n")

# Датасет 2
best_split_2 = tree.get_best_split(X2, y2)
print(f"Best split: Feature = {best_split_2['col_name']}, Split Value = {best_split_2['split_value']}, Gain = {best_split_2['gain']}\n")


Best split: Feature = feature_1, Split Value = 0.24665448467044643, Gain = 4243.420812246212

Best split: Feature = feature_2, Split Value = 0.28855607239980435, Gain = 5282.24536637628



In [5]:
# Тестирование класса
tree1 = MyTreeReg()
tree2 = MyTreeReg(max_depth = 10, max_leafs = 30)
tree3 = MyTreeReg(max_leafs = 25, min_samples_split = 15)

# Проверка
print(tree1)
print(tree2)
print(tree3)

MyTreeReg class: max_depth=5, min_samples_split=2, max_leafs=20
MyTreeReg class: max_depth=10, min_samples_split=2, max_leafs=30
MyTreeReg class: max_depth=5, min_samples_split=15, max_leafs=25
